## 환경설정

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### 1. 문서 로드(Load Documents)

In [2]:
!pip install PyMuPDF

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader

#PYMUPDFLoader 객체 정의
loader = PyMuPDFLoader("data/snow-white.pdf")

# 문서 로드
docs = loader.load()

print(f"문서의 페이지 수 : {len(docs)}")

문서의 페이지 수 : 6


In [4]:
print(docs[5].page_content)

왕자는깨어난백설공주를보고기뻐했어요.
“공주님, 나는이웃나라왕자입니다.”
“왕자님이나를다시살려주셨군요.”
“나와결혼해주시겠어요?”
“네, 좋아요!”
두사람은일곱난쟁이와함께오래오래행복하게살
았답니다.



In [5]:
print(docs[0].__dict__)

{'id': None, 'metadata': {'source': 'data/snow-white.pdf', 'file_path': 'data/snow-white.pdf', 'page': 0, 'total_pages': 6, 'format': 'PDF 1.5', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationDate': "D:20230912112024+09'00'", 'modDate': "D:20230912112024+09'00'", 'trapped': ''}, 'page_content': '백설공주\n옛날어느왕국에공주님이태어났어요.\n“어쩜이렇게어여쁠까? 살결이눈처럼하얗구나. 백\n설공주라고불러야겠다.”\n왕과왕비는갓태어난딸을보며기뻐했어요.\n하지만기쁨도잠시, 왕비는곧세상을떠나고말았어\n요.\n', 'type': 'Document'}


### 2. 문서 분할(Split Documents)

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap = 10)

split_documents = text_splitter.split_documents(docs)

print(f"분할된 청크의 수 : {len(split_documents)}")

분할된 청크의 수 : 21


### 3. 임베딩(Embedding 생성)

In [7]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

4. DB 생성(백스토어 생성) 및 저장
- FAISS(Facebook AI Similarity Search)
    - 페이스북에서 개발한 유사도 검색 및 클러스터링 라이브러리
    - 백터 데이터셋에서 빠른 유사도 검색

In [8]:
!pip install faiss-cpu

In [9]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [10]:
for doc in vectorstore.similarity_search("엄마"):
    print(doc.page_content)

“우아, 공주님이살아났어!”
어어? 이상하다! 모든게일곱. 의자도일곱, 접시도일곱. 어머, 
침대도일곱개네.”
도망치느라치진백설공주는식탁위에있던빵을먹고나서
일곱번째침대에쓰러져잠들었어요.
백설공주
옛날어느왕국에공주님이태어났어요.
“어쩜이렇게어여쁠까? 살결이눈처럼하얗구나. 백
설공주라고불러야겠다.”
왕과왕비는갓태어난딸을보며기뻐했어요.
“네, 좋아요!”
두사람은일곱난쟁이와함께오래오래행복하게살
았답니다.


### 5. 검색기(Retriever) 생성

In [11]:
# 백터스토어에 있는 정보를 검색하고 생성

retriver = vectorstore.as_retriever()
retriver.invoke("백설공주와 일곱난쟁이는 어디서 만났어?")

[Document(metadata={'source': 'data/snow-white.pdf', 'file_path': 'data/snow-white.pdf', 'page': 0, 'total_pages': 6, 'format': 'PDF 1.5', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationDate': "D:20230912112024+09'00'", 'modDate': "D:20230912112024+09'00'", 'trapped': ''}, page_content='백설공주\n옛날어느왕국에공주님이태어났어요.\n“어쩜이렇게어여쁠까? 살결이눈처럼하얗구나. 백\n설공주라고불러야겠다.”\n왕과왕비는갓태어난딸을보며기뻐했어요.'),
 Document(metadata={'source': 'data/snow-white.pdf', 'file_path': 'data/snow-white.pdf', 'page': 4, 'total_pages': 6, 'format': 'PDF 1.5', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationDate': "D:20230912112024+09'00'", 'modDate': "D:20230912112024+09'00'", 'trapped': ''}, page_content='저녁이되자, 일곱난쟁이가돌아왔어요.\n난쟁이들은쓰러진백설공주를보고엉엉울었어요.\n백설공주는깊은잠에빠진것처럼보였지요.\n“백설공주님, 못된왕비의꾐에넘어갔군요.

### 6. 프롬프트 생성

- 당신은 질문-답변 작업을 위한 어시스턴트입니다.
- 주어진 문맥을 사용하여 질문에 대답하세요
- 유치원 선생님이 아이에게 말하는 것처럼 매우 친절하고 부드러운 어조를 사용하세요
- 따뜻하고 친절한 방식으로 말하세요
- 답을 모르는 경우에는 모른다고 말하세요
- 한국어로 대답하세요

In [12]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question.
Use a very kind and gentle tone like a kindergarten teacher talking to a child.
Speak in a warm and friendly way.
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""

)

### 7. LLM 모델 생성

In [13]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0)

### 8. Chain 생성

In [14]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
# RunnablePassthrough -> 그대로 전달해주는 것

chain = (
    {"context": retriver, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
# 체인 실행
question = "백설공주가 먹은 사과에는 어떤 독이 있었어?"
response = chain.invoke(question)

print(response)

미안하지만, 백설공주가 먹은 사과에 어떤 독이 있었는지는 알 수 없어요. 하지만 사과를 먹고 백설공주가 정신을 잃고 쓰러졌다는 이야기가 있네요.


In [30]:
# 다시 해보기
from langchain_community.document_loaders import PyMuPDFLoader

#PYMUPDFLoader 객체 정의
loader = PyMuPDFLoader("data/SPRi-AI-Brief.pdf")

# 문서 로드
docs = loader.load()

print(f"문서의 페이지 수 : {len(docs)}")

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 30)

split_documents = text_splitter.split_documents(docs)

print(f"분할된 청크의 수 : {len(split_documents)}")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

retriver = vectorstore.as_retriever()
retriver.invoke("AI 시장에 대한 긍정적인 측면")

문서의 페이지 수 : 25
분할된 청크의 수 : 86


[Document(metadata={'source': 'data/SPRi-AI-Brief.pdf', 'file_path': 'data/SPRi-AI-Brief.pdf', 'page': 20, 'total_pages': 25, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13947', 'producer': 'Hancom PDF 1.3.0.547', 'creationDate': "D:20241007090546+09'00'", 'modDate': "D:20241007090546+09'00'", 'trapped': ''}, page_content='근로자의 41%는 일자리 시장이 나쁘다고 인식\n∙일자리 시장이 나쁘다고 인식하는 미국 근로자의 58%는 AI 발전으로 일자리가 줄어들 것으로 \n예상했으며, 일자리 시장이 좋다고 답변한 근로자 중에서는 일자리가 줄어들 것이라는 응답이 32%를 기록\n∙전체 근로자 중에서는 48%가 AI 발전으로 일자리가 줄어들 것이라고 답했으며, AI 발전으로 \n일자리가 늘어날 것이라는 응답은 11%, 영향이 없을 것이라는 응답은 28%를 기록\nn 미국 근로자의 27%는 직장에서 AI 도구를 주 1회 이상 사용하며, 전혀 사용하지 않는 비율은 49%를 기록\n∙2023년 7월에는 직장에서 AI 도구를 주 1회 이상 사용한다는 응답 비율이 20%였으며, 2024년 3월, \n8월에는 해당 응답이 각각 25%, 27%로 증가\nn 미국 근로자의 34%는 AI로 인한 실직이나 근무 시간 또는 급여가 줄어들 가능성을 우려하고 있으며, 이 \n수치는 2023년 7월 이래 비슷한 수준을 유지'),
 Document(metadata={'source': 'data/SPRi-AI-Brief.pdf', 'file_path': 'data/SPRi-AI-Brief.pdf', 'page': 22, 'total

In [31]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    """Say it like you're talking to a friend and throw in a bunch of jokes!
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""

)

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0)

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
# RunnablePassthrough -> 그대로 전달해주는 것

chain = (
    {"context": retriver, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 체인 실행
question = "AI시장이 커질수록 문제점은 뭐야? 어떤 페이지를 기반으로 답변하는건지 참고한 페이지도 함께 알려줘"
response = chain.invoke(question)

print(response)

친구야, AI 시장이 커지면 문제점이 뭐냐고? 음, 이건 마치 내가 다이어트한다고 하면서 매일 밤 치킨을 먹는 것과 비슷한 문제야. 겉으로는 좋아 보이지만, 속으로는 문제가 생길 수 있지! 😂

일단, AI가 발전하면서 일자리가 줄어들 거라는 걱정이 많아. 미국 근로자의 48%가 AI 때문에 일자리가 줄어들 거라고 생각한대. 이건 마치 내가 다이어트한다고 하면서도 치킨을 포기 못하는 것처럼, AI가 발전해도 일자리가 줄어드는 건 피할 수 없는 현실인 것 같아. (참고한 페이지: 20페이지)

그리고 유통, 제조업, 금융 서비스 분야에서도 일자리가 줄어들 거라고 하더라고. 하지만 반대로 AI 기술을 다루는 새로운 일자리가 생길 거라니, 이건 마치 치킨을 먹으면서도 운동을 열심히 해서 살을 빼는 것 같은 느낌이야! (참고한 페이지: 22페이지)

또한, AI가 발전하면서 실직이나 근무 시간, 급여가 줄어들 가능성에 대해 걱정하는 사람들도 많아. 하지만 63%는 낙관적이라니, 이건 마치 내가 치킨을 먹으면서도 내일은 꼭 운동할 거라고 다짐하는 것과 비슷한 것 같아. (참고한 페이지: 20페이지)

결론적으로, AI 시장이 커지면서 일자리 문제와 같은 걱정이 있지만, 새로운 기회도 생길 수 있다는 거지. 그러니 우리도 AI와 함께 잘 적응해보자고! 😄


In [7]:
!pip install bs4

ERROR: Could not find a version that satisfies the requirement SoupStrainerw (from versions: none)
ERROR: No matching distribution found for SoupStrainerw


In [1]:
import bs4
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 환경변수 가져오기
load_dotenv()

loader = WebBaseLoader(
    web_path=("https://n.news.naver.com/article/437/0000416134"),
    bs_kwargs=dict(
        # 특정 요소에서만 파싱하도록 제한하는 필터 적용
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class":["newsct_article _article_body", "media_and_head go_trans"]}
        )
    )
)

docs = loader.load()

# print(docs)

# 문서분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)

splits = text_splitter.split_documents(docs)

# 임베딩 생성
embedding = OpenAIEmbeddings()

# 백터 스토어 생성
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)

# 검색기 (retriver) 생성
retriver = vectorstore.as_retriever()

# runtime

# 프롬프트
prompt = PromptTemplate.from_template(
    """
    당신은 질문-답변을 수행하는 AI 어시스턴트이다.
    주어진 문맥에 검색된 다음문맥(context)를 사용해 질문에 답해야 한다.
    
    만약 주어진 문맥에서 답을 찾을 수 없는 경우, 모른다고 답해야 한다.
    주어진 질문에 대해서는 한글로 답변해야 한다.
    
    #Question
    {question}
    
    #Context
    {context}
    
    #Answer:
    """
)

# LLM 모델
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# chain 구성
news_chain = (
    {"context":retriver, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 질문
answer = news_chain.invoke("아파트는 몇 층이야?")
print(answer)



USER_AGENT environment variable not set, consider setting it to identify your requests.


주어진 문맥에서는 "아파트"가 로제와 브루노 마스가 함께 부른 노래 제목으로 언급되고 있습니다. 따라서 아파트의 층수에 대한 정보는 제공되지 않았습니다. 죄송하지만, 이 문맥에서는 아파트의 층수를 알 수 없습니다.
